In [ ]:
import cv2
import numpy as np

def ColorFinder(cap,x_min,x_max,y_min,y_max,out):
    
    return_value, frame = cap.read() # Obtain the frame of the video
    
    if return_value: # If an other frame exist
        hsv_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV) # Obtain the HSV value
        #height, width, _ = frame.shape (Can be used if you do it with a picture)
        score_color = [0,0,0,0,0,0,0] # List that will contain the amount of pixels of each color
        list_color = ['White','Black','Gray','Red','Blue','Green','Yellow'] # List of all Colors that can be detected
        list_color_BGR = [(255,255,255),(0,0,0),(128,128,128),(0,0,255),(255,0,0),(0,255,0),(0,255,255)] # List of all Colors that can be detected in BGR

        # Scan the area
        for i in range(x_min,x_max):
            for j in range(y_min,y_max):

                cx = i
                cy = j

                pixel_center = hsv_frame[cy, cx] # Get the HSV values of the pixel
                hue_value = pixel_center[0] # Get the Hue
                saturation_value = pixel_center[1] # Get the saturation
                value_value = pixel_center[2] # Get the value

                #White    
                if(saturation_value <= 30 and value_value >= 240): #White
                    score_color[0] = score_color[0] + 1 
                #Black    
                elif(value_value <= 70 and saturation_value <= 70): #Black
                    score_color[1] = score_color[1] + 1
                #Gray    
                elif((saturation_value<=10 and (value_value>=100 and value_value<=180)) or ((saturation_value>10 and saturation_value<=50) and (value_value>=60 and value_value<115))):
                    score_color[2] = score_color[2] + 1

                #Red
                elif((hue_value<=10 or (hue_value >=150 and hue_value <=180)) and (saturation_value >= 70 and value_value >= 80)):
                    score_color[3] = score_color[3] + 1

                #Blue    
                elif(hue_value >= 85 and hue_value <= 115 and (saturation_value >= 70 and value_value >= 80)):
                    score_color[4] = score_color[4] + 1

                #Green 
                elif(hue_value >= 35 and hue_value <= 80 and (saturation_value >= 70 and value_value >= 80)):  
                    score_color[5] = score_color[5] + 1

                #Yellow    
                elif(hue_value >= 15 and hue_value <= 32 and ((saturation_value >= 30 and value_value >= 230) or (saturation_value >= 80 and value_value >= 140))):
                    score_color[6] = score_color[6] + 1



        max_value = max(score_color) # Take the highest score
        max_index = score_color.index(max_value) # Take the position of this score
        ColorDetect = list_color[max_index] # Take the color at the same position in this list
        box_color = list_color_BGR[max_index] # Take the color at the same position in this list
        
        
        
        
        
        # Optional, allows you to display the area that will be scanned and display the color detected
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), box_color, 2)
        cv2.putText(frame, str(ColorDetect),(x_min, (y_min-5)),0, 0.5, box_color,2)

        cv2.imshow("Frame",frame) # Display the image of the video   

        result = np.asarray(frame) # Convert the frame into numpy array
        ret, buffer = cv2.imencode('.jpg', result) # Convert (encode) the image format into streaming data
        frame = buffer.tobytes() # Return the frame as a bytes object
        out.write(result) # Adding the image to the video being created
        cv2.waitKey(1) # Continue the video
        return True
    else: # If the video is finished or if a problem arises
        cap.release()
        cv2.destroyAllWindows()
        print('Video has ended or failed, try a different video format!')
        return False

def main():
    path = 'data/video/Parrot.mp4'
    cap = cv2.VideoCapture(path) # Using the computer camera (0) or video ('video name')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # Define frame width (Only for video)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))# Define frame Height (Only for video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    output_format = 'mp4v'
    codec = cv2.VideoWriter_fourcc(*output_format) # Define fps
    output = "data/output/VideoOutput.mp4"
    out = cv2.VideoWriter(output, codec, fps, (width, height))
    
    # Defining the area to be scanned
    x_min = 500
    x_max = 550
    y_min = 330
    y_max = 360
    
    # As long as the user does not stop the program with ctrl+c and there are other frames
    Output_video = True
    while (Output_video==True):
        Output_video = ColorFinder(cap,x_min,x_max,y_min,y_max,out) # Invoking the function
    
    cap.release()
    cv2.destroyAllWindows()
    
if __name__ == "__main__":
    main()